**Bushra Hoteit**

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [3]:
try:
  if not os.path.exists("historical_stocks.csv") or not os.path.exists("historical_stock_prices.csv"):
    raise FileNotFoundError("CSV files not found. Please ensure it is in your working directory.")
  else:
    stocks=pd.read_csv("historical_stocks.csv", encoding='utf-8')
    stock_prices=pd.read_csv("historical_stock_prices.csv", encoding='utf-8')
  if stocks.empty or stock_prices.empty:
    raise ValueError("One or both of the files loaded are empty. Please check the data.")
  print("Successfully loaded both datasets!")

except Exception as e:
  print(f"Error loading data: {str(e)}")
  raise


Successfully loaded both datasets!


In [4]:
stocks.head()

,ticker,exchange,name,sector,industry
0,PIH,NASDAQ,"1347 PROPERTY INSURANCE HOLDINGS, INC.",FINANCE,PROPERTY-CASUALTY INSURERS
1,PIHPP,NASDAQ,"1347 PROPERTY INSURANCE HOLDINGS, INC.",FINANCE,PROPERTY-CASUALTY INSURERS
2,TURN,NASDAQ,180 DEGREE CAPITAL CORP.,FINANCE,FINANCE/INVESTORS SERVICES
3,FLWS,NASDAQ,"1-800 FLOWERS.COM, INC.",CONSUMER SERVICES,OTHER SPECIALTY STORES
4,FCCY,NASDAQ,1ST CONSTITUTION BANCORP (NJ),FINANCE,SAVINGS INSTITUTIONS


In [5]:
# Convert date to date format

stock_prices['date'] = pd.to_datetime(stock_prices['date'])

In [6]:
stock_prices.head()

,ticker,open,close,adj_close,low,high,volume,date
0,AHH,11.50,11.58,8.493155,11.25,11.68,4633900,2013-05-08
1,AHH,11.66,11.55,8.471151,11.50,11.66,275800,2013-05-09
2,AHH,11.55,11.60,8.507822,11.50,11.60,277100,2013-05-10
3,AHH,11.63,11.65,8.544494,11.55,11.65,147400,2013-05-13
4,AHH,11.60,11.53,8.456484,11.50,11.60,184100,2013-05-14


In [7]:
# Merging the 2 files
df = pd.merge(stock_prices, stocks, on='ticker', how ='left')

# Set date as the index
df.set_index('date', inplace=True)
df.sort_index(inplace=True)
df.head()

,ticker,open,close,adj_close,low,high,volume,exchange,name,sector,industry
date,,,,,,,,,,,
1970-01-02,XOM,1.929688,1.937500,0.025623,1.925781,1.937500,1174400,NYSE,EXXON MOBIL CORPORATION,ENERGY,INTEGRATED OIL COMPANIES
1970-01-02,AA,7.140915,7.140915,1.986261,7.140915,7.225020,22500,NYSE,ALCOA CORPORATION,BASIC INDUSTRIES,ALUMINUM
1970-01-02,BA,0.925926,0.979424,0.229325,0.925926,0.979424,634400,NYSE,BOEING COMPANY (THE),CAPITAL GOODS,AEROSPACE
1970-01-02,DIS,0.688281,0.683144,0.282199,0.683144,0.689565,1109700,NYSE,WALT DISNEY COMPANY (THE),CONSUMER SERVICES,TELEVISION SERVICES
1970-01-02,PG,1.710938,1.718750,0.020346,1.708984,1.722656,832000,NYSE,PROCTER & GAMBLE COMPANY (THE),BASIC INDUSTRIES,PACKAGE GOODS/COSMETICS


In [8]:
# Create a decade column

df['decade'] = df.index.year // 10 * 10
df.head()

,ticker,open,close,adj_close,low,high,volume,exchange,name,sector,industry,decade
date,,,,,,,,,,,,
1970-01-02,XOM,1.929688,1.937500,0.025623,1.925781,1.937500,1174400,NYSE,EXXON MOBIL CORPORATION,ENERGY,INTEGRATED OIL COMPANIES,1970
1970-01-02,AA,7.140915,7.140915,1.986261,7.140915,7.225020,22500,NYSE,ALCOA CORPORATION,BASIC INDUSTRIES,ALUMINUM,1970
1970-01-02,BA,0.925926,0.979424,0.229325,0.925926,0.979424,634400,NYSE,BOEING COMPANY (THE),CAPITAL GOODS,AEROSPACE,1970
1970-01-02,DIS,0.688281,0.683144,0.282199,0.683144,0.689565,1109700,NYSE,WALT DISNEY COMPANY (THE),CONSUMER SERVICES,TELEVISION SERVICES,1970
1970-01-02,PG,1.710938,1.718750,0.020346,1.708984,1.722656,832000,NYSE,PROCTER & GAMBLE COMPANY (THE),BASIC INDUSTRIES,PACKAGE GOODS/COSMETICS,1970


**1. Advanced Data Cleaning**

***Handle Missing Values: Employ advanced imputation techniques where appropriate, such as interpolation, using backward or forward filling, or model-based imputation.***

In [9]:
# Missing values in merged file

print("Missing values in merged file:\n", df.isnull().sum())

Missing values in merged file:
 ticker             0
open               0
close              0
adj_close          0
low                0
high               0
volume             0
exchange           0
name               0
sector       2549449
industry     2549449
decade             0
dtype: int64


In [10]:
# Filling missing values under sector & industry columns with 'Unknown'

df['sector'] = df['sector'].fillna('Unknown')
df['industry'] = df['industry'].fillna('Unknown')

In [11]:
# In case we had missing values for prices we can use forward fill

price_columns = ['open', 'close', 'low', 'high']

df[price_columns] = df[price_columns].ffill()

In [12]:
# Missing values after cleaning

print("Updated missing values in merged file:\n", df.isnull().sum())

Updated missing values in merged file:
 ticker       0
open         0
close        0
adj_close    0
low          0
high         0
volume       0
exchange     0
name         0
sector       0
industry     0
decade       0
dtype: int64


***Detect and Resolve Outliers: Apply robust methods to detect outliers in critical variables such as close prices and volume. Decide on a strategy to manage these outliers, whether it be removal, capping, or correction.***

In [13]:
# Detecting outliers using IQR

Q1 = df['close'].quantile(0.25)
Q3 = df['close'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['close'] < lower_bound) | (df['close'] > upper_bound)]
outliers

,ticker,open,close,adj_close,low,high,volume,exchange,name,sector,industry,decade
date,,,,,,,,,,,,
1970-01-02,NAV,251.250000,261.250000,169.416168,251.250000,261.250000,900,NYSE,NAVISTAR INTERNATIONAL CORPORATION,CAPITAL GOODS,AUTO MANUFACTURING,1970
1970-01-05,NAV,262.500000,273.750000,177.522263,262.500000,277.500000,1700,NYSE,NAVISTAR INTERNATIONAL CORPORATION,CAPITAL GOODS,AUTO MANUFACTURING,1970
1970-01-06,NAV,273.750000,275.000000,178.332855,270.000000,277.500000,1200,NYSE,NAVISTAR INTERNATIONAL CORPORATION,CAPITAL GOODS,AUTO MANUFACTURING,1970
1970-01-07,NAV,275.000000,271.250000,175.901031,271.250000,276.250000,700,NYSE,NAVISTAR INTERNATIONAL CORPORATION,CAPITAL GOODS,AUTO MANUFACTURING,1970
1970-01-08,NAV,271.250000,271.250000,175.901031,270.000000,273.750000,700,NYSE,NAVISTAR INTERNATIONAL CORPORATION,CAPITAL GOODS,AUTO MANUFACTURING,1970
...,...,...,...,...,...,...,...,...,...,...,...,...
2018-08-24,CHDN,283.850006,282.700012,282.700012,281.250000,288.000000,51300,NASDAQ,"CHURCHILL DOWNS, INCORPORATED",CONSUMER SERVICES,SERVICES-MISC. AMUSEMENT & RECREATION,2010
2018-08-24,MSTR,142.399994,149.669998,149.669998,142.350006,150.080002,224900,NASDAQ,MICROSTRATEGY INCORPORATED,TECHNOLOGY,COMPUTER SOFTWARE: PREPACKAGED SOFTWARE,2010
2018-08-24,ADI,98.059998,98.779999,98.779999,97.440002,98.889999,2674800,NASDAQ,"ANALOG DEVICES, INC.",TECHNOLOGY,SEMICONDUCTORS,2010


In [14]:
# Cap outliers for 'close' prices

df['close'] = np.where(df['close'] < lower_bound, lower_bound, df['close'])
df['close'] = np.where(df['close'] > upper_bound, upper_bound, df['close'])

print("Close price outliers capped at:")
print(f"Lower bound: {lower_bound:.2f}, Upper bound: {upper_bound:.2f}")

Close price outliers capped at:
Lower bound: -25.83, Upper bound: 63.05


***Error Identification and Correction: Scan the dataset for errors introduced either through data entry or during initial data collection. Correct these errors to ensure data integrity.***

In [15]:
# Correcting illogical values such as negative stock prices

df['close'] = df['close'].apply(lambda x: abs(x) if x < 0 else x)
df['volume'] = df['volume'].apply(lambda x: abs(x) if x < 0 else x)

In [16]:
# Removing duplicates

df.drop_duplicates(inplace=True)
print(f"Duplicates: {df.duplicated().sum()}")

Duplicates: 0


**2. Data Transformation**

***Feature Engineering: Create new features that could enhance the model’s predictive power. This includes rolling averages, volatility measures, and technical indicators***

In [17]:
# Rolling averages

df['rolling_avg_close'] = df['close'].rolling(window=20).mean()


In [18]:
# Volatility (standard deviation over the past 30 days)

df['volatility'] = df['close'].rolling(window=30).std()

In [19]:
# Technical indicators (RSI)

delta = df['close'].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
rs = gain / loss
df['RSI'] = 100 - (100 / (1 + rs))


In [20]:
# View of the new features

df[['ticker','close', 'rolling_avg_close', 'volatility', 'RSI']].tail()

,ticker,close,rolling_avg_close,volatility,RSI
date,,,,,
2018-08-24,INST,39.099998,34.3895,19.991550,54.370580
2018-08-24,SQQQ,11.890000,32.2495,20.020159,44.732286
2018-08-24,PBFX,21.600000,31.7345,19.908982,46.242125
2018-08-24,POPE,63.049998,34.0800,19.971487,52.008686
2018-08-24,NZF,14.690000,31.6620,20.024798,43.849206


***Data Normalization/Standardization: Standardize or normalize numerical fields to ensure that model inputs have similar scales.***

In [21]:
# Standardizing close & volume columns

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

cols = ['open', 'close', 'high', 'low', 'volume']

df[cols] = scaler.fit_transform(df[cols])

***Encoding Categorical Variables: Apply appropriate encoding techniques such as one-hot encoding for categorical data.***

In [22]:
print(df['sector'].nunique(), "unique sectors")
print(df['industry'].nunique(), "unique industries")
print(df['exchange'].nunique(), "unique exchanges")


13 unique sectors
136 unique industries
2 unique exchanges


In [23]:
from sklearn.preprocessing import LabelEncoder
# I will use label encoder instead of one hot encoding due to RAM limit

# Initialize label encoder
le = LabelEncoder()

# Columns to encode
cols_to_encode = ['sector', 'industry', 'exchange']

# Encode each column and add back as new columns
for col in cols_to_encode:
    df[col + '_encoded'] = le.fit_transform(df[col].astype(str))


In [24]:
# View of the encoded variables

df[['ticker','sector', 'industry', 'exchange', 'sector_encoded', 'industry_encoded', 'exchange_encoded']]

,ticker,sector,industry,exchange,sector_encoded,industry_encoded,exchange_encoded
date,,,,,,,
1970-01-02,XOM,ENERGY,INTEGRATED OIL COMPANIES,NYSE,5,64,1
1970-01-02,AA,BASIC INDUSTRIES,ALUMINUM,NYSE,0,5,1
1970-01-02,BA,CAPITAL GOODS,AEROSPACE,NYSE,1,2,1
1970-01-02,DIS,CONSUMER SERVICES,TELEVISION SERVICES,NYSE,4,127,1
1970-01-02,PG,BASIC INDUSTRIES,PACKAGE GOODS/COSMETICS,NYSE,0,97,1
...,...,...,...,...,...,...,...
2018-08-24,INST,TECHNOLOGY,COMPUTER SOFTWARE: PREPACKAGED SOFTWARE,NYSE,10,30,1
2018-08-24,SQQQ,Unknown,Unknown,NASDAQ,12,133,0
2018-08-24,PBFX,ENERGY,NATURAL GAS DISTRIBUTION,NYSE,5,85,1


**3. Integration and Formatting for Modeling**

***Consolidate Data: Ensure all data transformations and cleanings are integrated into a single, coherent dataset ready for analysis.***

In [25]:
# Cleaned and transformed data to be used for analysis
df.tail()

,ticker,open,close,adj_close,low,high,volume,exchange,name,sector,industry,decade,rolling_avg_close,volatility,RSI,sector_encoded,industry_encoded,exchange_encoded
date,,,,,,,,,,,,,,,,,,
2018-08-24,INST,-0.013327,0.960307,39.099998,-0.013160,-0.013014,-0.075242,NYSE,"INSTRUCTURE, INC.",TECHNOLOGY,COMPUTER SOFTWARE: PREPACKAGED SOFTWARE,2010,34.3895,19.991550,54.370580,10,30,1
2018-08-24,SQQQ,-0.022456,-0.527601,11.890000,-0.022723,-0.022006,0.895560,NASDAQ,PROSHARES ULTRAPRO SHORT QQQ,Unknown,Unknown,2010,32.2495,20.020159,44.732286,12,133,0
2018-08-24,PBFX,-0.019067,0.003365,21.600000,-0.019225,-0.018769,-0.086860,NYSE,PBF LOGISTICS LP,ENERGY,NATURAL GAS DISTRIBUTION,2010,31.7345,19.908982,46.242125,5,85,1
2018-08-24,POPE,-0.001147,2.269950,73.000000,-0.000520,-0.001752,-0.093366,NASDAQ,POPE RESOURCES,CONSUMER NON-DURABLES,ENVIRONMENTAL SERVICES,2010,34.0800,19.971487,52.008686,3,48,0
2018-08-24,NZF,-0.021588,-0.374491,14.690000,-0.021735,-0.021151,-0.079748,NYSE,NUVEEN MUNICIPAL CREDIT INCOME FUND,Unknown,Unknown,2010,31.6620,20.024798,43.849206,12,133,1


***Data Splitting: Prepare the data for predictive modeling by splitting it into training, validation, and test sets.***

In [ ]:
from sklearn.model_selection import train_test_split

# Features in X, target in y
X = df.drop(columns=['close', 'ticker', 'name', 'exchange', 'sector', 'industry', 'decade'])  # I kept the price columns, new added features & encoded variables
y = df['close']     # To predict the closing prices

# First split: train + temp (temp = val+test)
# X_train is 70% of the data
# X_temp is 30% of the data which will be split into val & test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

# Second split: validation + test (Splitting X_temp & t_temp into validation & test set)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, shuffle=True)


***Save clean data: Save cleaned data and the splits for future use.***

In [ ]:
# Save full cleaned dataset
df.to_csv('cleaned_stock_data.csv', index=False)

# Save splits
X_train.to_csv('X_train.csv', index=False)
y_train.to_csv('y_train.csv', index=False)
X_val.to_csv('X_val.csv', index=False)
y_val.to_csv('y_val.csv', index=False)
X_test.to_csv('X_test.csv', index=False)
y_test.to_csv('y_test.csv', index=False)
